#Coarse section

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
# Your file will probably be called this, if you used the regular Kaggle download.
!unzip -q /gdrive/MyDrive/ML_project_privateTest/living-thing-classification-coarse-private.zip     # CHANGE DIRECTORY

In [ ]:
from pathlib import Path

In [ ]:
batch_size = 64
img_size = 112
fine_grain = False
num_classes = 50 if fine_grain else 8

root = Path('/content/privtest_coarse')

In [ ]:
!ls -alh privtest_coarse

total 204K
drwxr-xr-x 3 root root 4.0K May 29 08:42 .
drwxr-xr-x 1 root root 4.0K May 29 08:42 ..
drwxr-xr-x 2 root root 124K May 29 08:42 privtest
-rw-r--r-- 1 root root  72K May 27 09:19 privtest_coarse.csv


In [ ]:
import tensorflow as tf
import pandas as pd
import json
from pathlib import Path

def load_target_names(path='public/info.json'):
    with open(path) as f:
        info = json.load(f)
    return info['fine'], info['coarse']

def load_image(file_path, image_size=112):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, [image_size, image_size])
    return img

root = Path('/content/privtest_coarse')  # Update the root directory path

def create_dataset_tf(split, fine_grained, batch_size=32, image_size=112):
    df = pd.read_csv(root / f"{split}.csv")
    file_paths = df.filename.map(lambda fn: str(root / "privtest" / fn))
    labels = df.fine if fine_grained else df.coarse

    dataset = tf.data.Dataset.from_tensor_slices((file_paths, labels))
    dataset = dataset.map(lambda x, y: (load_image(x, image_size=image_size), y))
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset

In [ ]:
from tensorflow.keras.models import load_model

model = load_model('/gdrive/MyDrive/ML_project_coarse_inceptionv3_model/coarseInception.h5')

In [ ]:
private_test_ds = create_dataset_tf('privtest_coarse', fine_grained=fine_grain, batch_size=batch_size)

In [ ]:
# Predict with the loaded model
y_pred_coarse = model.predict(private_test_ds)

24/24 [==============================] - 51s 2s/step


In [ ]:
import pandas as pd
import numpy as np
y_pred=np.argmax(y_pred_coarse, axis=1)

In [ ]:
test_csv = pd.read_csv(root / f"privtest_coarse.csv")
test_csv['coarse']=y_pred
pred_file="/gdrive/MyDrive/ML_project_coarse_preds/47541164_deep_coarse.csv"
test_csv.to_csv(pred_file, index=False)
print("Predictions saved succesfully at %s" % pred_file)

Predictions saved succesfully at /gdrive/MyDrive/ML_project_coarse_preds/47541164_deep_coarse.csv


#Fine section

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
# Your file will probably be called this, if you used the regular Kaggle download.
!unzip -q /gdrive/MyDrive/ML_project_privateTest/living-thing-classification-fine-private.zip     # CHANGE DIRECTORY

In [ ]:
from pathlib import Path

In [ ]:
batch_size = 64
img_size = 224
fine_grain = True
num_classes = 50 if fine_grain else 8

root = Path('/content/privtest_fine')

In [ ]:
!ls -alh privtest_fine

total 204K
drwxr-xr-x 3 root root 4.0K May 29 10:14 .
drwxr-xr-x 1 root root 4.0K May 29 10:14 ..
drwxr-xr-x 2 root root 124K May 29 10:14 privtest
-rw-r--r-- 1 root root  72K May 27 09:30 privtest_fine.csv


In [ ]:
import tensorflow as tf
import pandas as pd
import json
from pathlib import Path

def load_target_names(path='public/info.json'):
    with open(path) as f:
        info = json.load(f)
    return info['fine'], info['coarse']

def load_image(file_path, image_size=112):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, [image_size, image_size])
    return img

root = Path('/content/privtest_fine')  # Update the root directory path

def create_dataset_tf(split, fine_grained, batch_size=32, image_size=112):
    df = pd.read_csv(root / f"{split}.csv")
    file_paths = df.filename.map(lambda fn: str(root / "privtest" / fn))
    labels = df.fine if fine_grained else df.coarse

    dataset = tf.data.Dataset.from_tensor_slices((file_paths, labels))
    dataset = dataset.map(lambda x, y: (load_image(x, image_size=image_size), y))
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset

In [ ]:
from tensorflow.keras.models import load_model

model = load_model('/gdrive/MyDrive/ML_project_fine_mobileNet_model/fineMobileNet.h5')

In [ ]:
private_test_ds = create_dataset_tf('privtest_fine', fine_grained=True, batch_size=batch_size, image_size=224)

In [ ]:
# Predict with the loaded model
y_pred_fine = model.predict(private_test_ds)

24/24 [==============================] - 48s 2s/step


In [ ]:
import pandas as pd
import numpy as np
y_pred=np.argmax(y_pred_fine, axis=1)

In [ ]:
test_csv = pd.read_csv(root / f"privtest_fine.csv")
test_csv['fine']=y_pred
pred_file="/gdrive/MyDrive/ML_project_preds/47541164_deep_fine.csv"
test_csv.to_csv(pred_file, index=False)
print("Predictions saved succesfully at %s" % pred_file)

Predictions saved succesfully at /gdrive/MyDrive/ML_project_preds/47541164_deep_fine.csv
